In [33]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from joblib import Parallel, delayed
import disease_network_funs as fun_py
from importlib import reload
from itertools import chain

from sqlalchemy.dialects.mysql import DOUBLE
from sqlalchemy.types import Integer, String, Text

In [23]:
reload(fun_py)

<module 'disease_network_funs' from '/home/hashjamm/codes/disease_network/disease_network_funs.py'>

In [3]:
sas_path = "/home/hashjamm/project_data/disease_network/sas_files/"
edge_pids_path = "/home/hashjamm/results/disease_network/edge_pids/"
matched_path = "/home/hashjamm/project_data/disease_network/sas_files/matched/"
ctable_path = "/home/hashjamm/results/disease_network/ctables/"
pids_info_path = "/home/hashjamm/results/disease_network/"
final_results_path = "/home/hashjamm/results/disease_network/final_results/"
network_path = "/home/hashjamm/results/disease_network/default_network_properties/"
db_migration_path = "/home/hashjamm/results/disease_network/db_migration/"

In [5]:
files = [i for i in os.listdir(final_results_path) if i[:3] == 'cis']
sorted_files = sorted(files, key=lambda x: int(x.split('_')[-1].replace('.csv', '')))

In [6]:
results = Parallel(n_jobs=10)(
    delayed(fun_py.edge_stat_attr_maker)(filename) for filename in sorted_files
)

In [7]:
edge_stats_raw = []
edge_attrs_raw = []

for idx, (edge_stat, edge_attr) in enumerate(results):
    fu = idx + 1

    edge_stat = edge_stat.copy()
    edge_stat['fu'] = fu
    edge_stat = edge_stat[['fu'] + [col for col in edge_stat.columns if col != 'fu']]
    edge_stats_raw.append(edge_stat)

    edge_attrs_raw.append((fu, edge_attr.copy()))

In [14]:
edge_attrs_final_1 = Parallel(n_jobs=5)(
    delayed(fun_py.transform_edge_attr)(item) for item in edge_attrs_raw[:5]
)

In [14]:
edge_attrs_final_2 = Parallel(n_jobs=3)(
    delayed(fun_py.transform_edge_attr)(item) for item in edge_attrs_raw[5:8]
)

In [14]:
edge_attrs_final_3 = Parallel(n_jobs=2)(
    delayed(fun_py.transform_edge_attr)(item) for item in edge_attrs_raw[8:]
)

In [17]:
whole_edge_stat = pd.concat(edge_stats_raw, ignore_index=True)

In [18]:
edge_attrs_final = list(chain.from_iterable([
    edge_attrs_final_1,
    edge_attrs_final_2,
    edge_attrs_final_3
]))

whole_edge_attr = pd.concat(edge_attrs_final, ignore_index=True)

In [34]:
whole_edge_stat.to_csv(f'{db_migration_path}edge_stat.csv', index = False)
whole_edge_attr.to_csv(f'{db_migration_path}edge_attr.csv', index = False)

In [4]:
whole_edge_stat =\
pd.read_csv(f'{db_migration_path}edge_stat.csv', 
            dtype=\
            {
                'fu': int,
                'cause_abb': str,
                'outcome_abb': str,
                'ct00': int,
                'ct01': int,
                'ct10': int,
                'ct11': int,
                'rr_values': float,
                'rr_lower_cis': float,
                'rr_upper_cis': float,
                'log_rr_values': float,
                'chisq_values': float,
                'chisq_p_values': float,
                'fisher_p_values': float,
                'adjusted_chisq_p_values': float,
                'adjusted_fisher_p_values': float
            },
            low_memory=False)

In [ ]:
whole_edge_attr =\
pd.read_csv(f'{db_migration_path}edge_attr.csv',
            dtype=\
            {
                'fu': int,
                'cause_abb': str,
                'outcome_abb': str,
                'attribute_1': str,
                'value_1': int,
                'attribute_2': 'string',
                'value_2': 'Int64',
                'count': float
            },
            low_memory=False)

In [5]:
# DB 접속 정보
db_user = 'migrate_user'
db_pass = 'qkrhd20164231!'
db_name = 'cotdex_db'
host = 'localhost'       # 또는 서버 IP
port = 3306              # 기본 포트

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{db_user}:{db_pass}@{host}:{port}/{db_name}?charset=utf8mb4")

# Maria DB에서 root 계정은 auth_socket 설정으로, 외부 접근이 불가 -> 이를 최대한 수정하지 않으려고 노력함
# Maria DB에서 Django 서버에서 CRUD 권한만 부여하여 사용할 cotdex_django_user 계정은 cotdex_db에 CRUD 권한 부여 -> CREATE 불가
# 따라서 임시로 migrate_user 계정을 만들고, cotdex_db에 SELECT, INSERT, UPDATE, DELETE, CREATE, DROP 권한 부여 -> migrate 후 계정 삭제

In [6]:
# 컬럼 타입 매핑
dtype_map_stat = {
    'fu': Integer(),
    'cause_abb': String(10),
    'outcome_abb': String(10),
    'ct00': Integer(),
    'ct01': Integer(),
    'ct10': Integer(),
    'ct11': Integer(),
    'rr_values': DOUBLE(),
    'rr_lower_cis': DOUBLE(),
    'rr_upper_cis': DOUBLE(),
    'log_rr_values': DOUBLE(),
    'chisq_values': DOUBLE(),
    'chisq_p_values': DOUBLE(),
    'fisher_p_values': DOUBLE(),
    'adjusted_chisq_p_values': DOUBLE(),
    'adjusted_fisher_p_values': DOUBLE()
}

# 컬럼 타입 매핑
dtype_map_attr = {
    'fu': Integer(),
    'cause_abb': String(10),
    'outcome_abb': String(10),
    'attribute_1': String(20),
    'value_1': Integer(),
    'attribute_2': String(20),
    'value_2': Integer(),
    'count': Integer()
}

In [7]:
whole_edge_stat.to_sql(
    name='edge_stat',
    con=engine,
    if_exists='replace',   # 또는 'append'
    index=False,
    dtype=dtype_map_stat
)

In [11]:
whole_edge_attr.to_sql(
    name='edge_attr',
    con=engine,
    if_exists='replace',   # 또는 'append'
    index=False,
    dtype=dtype_map_attr
)

In [12]:
node_info = pd.read_csv(f'{pids_info_path}node_pids_info.csv')

In [14]:
node_info.head()

,node_code,width,height,sex_1_counts,sex_2_counts,age_0_counts,age_1_counts,age_2_counts,age_3_counts,age_4_counts,...,sex_2_ctrb_3_counts,sex_2_ctrb_4_counts,sex_2_ctrb_5_counts,sex_2_ctrb_6_counts,sex_2_ctrb_7_counts,sex_2_ctrb_8_counts,sex_2_ctrb_9_counts,sex_2_ctrb_10_counts,Korean,English
0,J03,1.000000,1.000000,41118,51040,1227,7231,11844,7787,5556,...,3706,4041,4719,5337,6200,6945,6932,6680,급성 편도염,Acute tonsillitis
1,J20,0.999158,0.999158,33557,44137,1392,7547,8334,5408,3702,...,3076,3551,4110,4759,5237,6008,5889,5823,급성 기관지염,Acute bronchitis
2,J06,0.998315,0.998315,30228,39585,1318,7861,7637,4603,3452,...,2858,3271,3662,4201,4824,5166,5258,5252,다발성 및 상세불명 부위의 급성 상기도 감염,Acute upper respiratory infections of muitiple...
3,J02,0.997473,0.997473,27607,36329,1561,7785,7644,4575,3204,...,2446,2923,3262,3855,4456,4892,5164,4934,급성 인두염,Acute pharyngitis
4,J00,0.996630,0.996630,27512,35232,1308,7750,6915,4203,3031,...,2525,2972,3344,3615,4196,4626,4698,4474,급성 비인두염[감기],Acute nasopharyngitis[common cold]


In [38]:
node_base = node_info[['node_code', 'width', 'height', 'Korean', 'English']]

In [39]:
node_base

,node_code,width,height,Korean,English
0,J03,1.000000,1.000000,급성 편도염,Acute tonsillitis
1,J20,0.999158,0.999158,급성 기관지염,Acute bronchitis
2,J06,0.998315,0.998315,다발성 및 상세불명 부위의 급성 상기도 감염,Acute upper respiratory infections of muitiple...
3,J02,0.997473,0.997473,급성 인두염,Acute pharyngitis
4,J00,0.996630,0.996630,급성 비인두염[감기],Acute nasopharyngitis[common cold]
...,...,...,...,...,...
1182,T37,0.014322,0.167650,기타 전신성 항감염제 및 항기생충제에 의한 중독,Poisoning by other systemic antiinfectives and...
1183,T54,0.038753,0.109520,부식물질의 독작용,Toxic effect of corrosive substances
1184,T75,0.014322,0.167650,기타 외인의 영향,Effects of other extenal causes
1185,T94,0.076664,0.085088,다발성 및 상세불명의 신체 부위를 침범한 손상의 후유증,Sequelae of injuries involving mulitple and un...


In [44]:
node_attr = fun_py.melting_node_attr(node_info)

In [45]:
node_attr

,node_code,attribute_1,value_1,attribute_2,value_2,count
0,J03,sex,1,None,None,41118
1,J20,sex,1,None,None,33557
2,J06,sex,1,None,None,30228
3,J02,sex,1,None,None,27607
4,J00,sex,1,None,None,27512
...,...,...,...,...,...,...
124536,K45,sex,2,ctrb,10,1
124537,L12,sex,2,ctrb,10,1
124538,Q64,sex,2,ctrb,10,1
124539,R72,sex,2,ctrb,10,1


In [46]:
# 컬럼 타입 매핑
dtype_map_base = {
    'node_code': String(10),
    'width': DOUBLE(),
    'heigth': DOUBLE(),
    'Korean': Text(),
    'English': Text()
}

# 컬럼 타입 매핑
dtype_map_attr = {
    'node_code': String(10),
    'attribute_1': String(20),
    'value_1': Integer(),
    'attribute_2': String(20),
    'value_2': Integer(),
    'count': Integer()
}

In [47]:
node_base.to_sql(
    name='node_base',
    con=engine,
    if_exists='replace',   # 또는 'append'
    index=False,
    dtype=dtype_map_base
)

In [48]:
node_attr.to_sql(
    name='node_attr',
    con=engine,
    if_exists='replace',   # 또는 'append'
    index=False,
    dtype=dtype_map_attr
)